In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def relu(Z):
    A = np.maximum(0, Z)
    activation_cache = Z
    return A, activation_cache


In [3]:
def sigmoid(Z):
    A = 1 / (1 + np.exp(-Z))
    activation_cache = Z
    return A, activation_cache


In [4]:

def initialize_parameters(n_x, n_h, n_y):
    np.random.seed(1)

    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))

    parameters = {
        "W1": W1,
        "b1": b1,
        "W2": W2,
        "b2": b2
    }

    return parameters

In [5]:
def initialize_parameters_deep(layer_dims):
    np.random.seed(1)
    parameters = {}
    L = len(layer_dims)  # number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) / np.sqrt(layer_dims[l-1])  # *0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))

    return parameters


In [6]:
def linear_forward(A, W, b):
    Z = np.dot(W, A) + b
    cache = (A, W, b)

    return Z, cache


In [7]:
def linear_activation_forward(A_prev, W, b, activation):
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
    elif activation == "tanh":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = tanh(Z)
    else:
        print("Invalid activation function")
        return

    cache = (linear_cache, activation_cache)

    return A, cache


In [8]:
def L_model_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2  # number of layers in the neural network
    
    # Implement [LINEAR -> ACTIVATION]*(L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters["W" + str(l)], parameters["b" + str(l)], activation="relu")
        caches.append(cache)

    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    AL, cache = linear_activation_forward(A, parameters["W" + str(L)], parameters["b" + str(L)], activation="sigmoid")
    caches.append(cache)

    return AL, caches

In [9]:
def compute_cost(AL, Y,epsilon=1e-8):
    m = Y.shape[1]
    cost = -(1/m) * np.sum(Y * np.log(AL + epsilon))
    return cost

In [10]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = (1/m) * np.dot(dZ, A_prev.T)
    db = (1/m) * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)

    return dA_prev, dW, db


In [11]:
def sigmoid_backward(dA, activation_cache):
    Z = activation_cache
    s = 1 / (1 + np.exp(-Z))
    dZ = dA * s * (1 - s)
    return dZ


In [12]:
def relu_backward(dA, activation_cache):
    Z = activation_cache
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0
    return dZ

In [13]:
def tanh_backward(dA, activation_cache):
    Z = activation_cache
    s = np.tanh(Z)
    dZ = dA * (1 - s**2)
    return dZ

In [14]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache

    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    elif activation == "tanh":
        dZ = tanh_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    else:
        print("Invalid activation function")
        return

    return dA_prev, dW, db


In [15]:
def L_model_backward(AL, Y, caches, epsilon=1e-8):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    dAL = - (np.divide(Y, AL + epsilon) - np.divide(1 - Y, 1 - AL + epsilon))
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "AL, Y, caches". Outputs: "grads["dAL"], grads["dWL"], grads["dbL"]
    current_cache = caches[L-1]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation="sigmoid")

    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 2)], current_cache, activation="relu")
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads


In [16]:
def update_parameters(params, grads, learning_rate):
    L = len(params) // 2  # number of layers in the neural network

    # Update rule for each parameter
    for l in range(L):
        params["W" + str(l+1)] = params["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        params["b" + str(l+1)] = params["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]

    return params


In [17]:
def L_layer_model(X, Y, layers_dims, learning_rate=0.0075, num_iterations=3000, print_cost=False):
    np.random.seed(1)
    costs = []  # keep track of cost

    # Parameters initialization.
    parameters = initialize_parameters_deep(layers_dims)

    # Loop (gradient descent)
    for i in range(0, num_iterations):

        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
        AL, caches = L_model_forward(X, parameters)

        # Compute cost.
        cost = compute_cost(AL, Y)

        # Backward propagation.
        grads = L_model_backward(AL, Y, caches)

        # Update parameters.
        parameters = update_parameters(parameters, grads, learning_rate)
#         print("Cost after iteration %i: %f" % (i, cost))
        # Print the cost every 100 training example
        if print_cost and i % 10 == 0:
            print("Cost after iteration %i: %f" % (i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)

    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()

    return parameters,costs


In [28]:
df = pd.read_csv("/Users/mandeez/Downloads/Updated_Dataset.csv",encoding='ISO-8859-1',delimiter=";")

In [31]:
df.head()

,Bitki_Türü,Tarih,Hour,380_440,400_460,420_480,460_520,480_540,510_570,530_590,...,670_730,700_760,730_790,780_840,820_880,860_920,900_960,SICAKLIK,NEM,Fruit Numeric
0,Topalak,43578,0.710417,424,431,443,486,517,543,563,...,690,734,789,829,839,888,939,24,163,10
1,Topalak,43467,0.770833,404,442,451,483,514,526,552,...,685,736,783,829,844,897,939,20,176,10
2,Ayrik_Otu,43721,0.831250,383,409,446,507,495,535,575,...,708,722,749,785,875,916,941,20,169,0
3,Kanyas,43697,0.952083,408,431,447,471,502,513,545,...,696,712,774,812,872,838,932,25,96,5
4,Sirken,43629,0.012500,403,423,440,495,515,541,556,...,699,735,743,805,853,889,954,26,107,9


In [30]:
from sklearn.preprocessing import LabelEncoder
#Encode bitki_turu to Numeric
label_encoder = LabelEncoder()
df['Fruit Numeric'] = label_encoder.fit_transform(df['Bitki_Türü'])

In [33]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
# Extract the features and labels
X = df.drop(["Bitki_Türü",'Fruit Numeric','Tarih','Hour'], axis=1)
y = df['Fruit Numeric']

# Initialize the scalers
min_max_scaler = MinMaxScaler()
std_scaler = StandardScaler()

# Scale the data using MinMaxScaler
X_scaled_minmax = min_max_scaler.fit_transform(X)

# Standardize the scaled data using StandardScaler
X_scaled_std = std_scaler.fit_transform(X_scaled_minmax)

In [34]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled_std, y, test_size=0.2, random_state=42)

In [51]:
y_train[1]

10

In [42]:
layers_dims = [8000,20,100,10]

In [50]:
L_layer_model(X_train,y_train,layers_dims,print_cost=True)

IndexError: tuple index out of range

In [21]:
def predict(X, parameters):
    AL, _ = L_model_forward(X, parameters)
    Y_pred = np.argmax(AL, axis=0)
    return Y_pred

In [22]:
def accuracy(Y_pred, Y):
    num_correct = np.sum(Y_pred == Y)
    print(num_correct)
    accuracy = num_correct / Y.shape[0]
    return accuracy